# [실습1] LangChain으로 간단한 LLM 챗봇 만들기 1


## 실습 목표
---
- LangChain을 활용해서 gpt-4o-mini 모델을 사용하는 챗봇을 개발합니다.
- 짧은 Chain을 구성하고, 이를 활용해서 챗봇을 구현합니다.

## 실습 목차
---

1. **ChatOpenAI Agent 생성:** 사용자의 입력에 대한 ChatGPT의 gpt-4o-mini 모델의 답변을 받아오는 Agent를 생성합니다.

2. **챗봇 Chain 구성**: ChatOpenAI Agent를 비롯하여 챗봇 구현에 필요한 Agent들을 엮어서 챗봇 Chain으로 구성합니다.

3. **챗봇 사용**: 여러분이 구성하신 챗봇을 사용해봅니다.

## 실습 개요
---

LangChain의 Chain을 활용해서 gpt-4o-mini 모델을 활용하는 챗봇을 구현하고, Chain을 형성하는 방법을 이해합니다.


## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.

In [1]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama, ChatOpenAI

In [2]:
import os


if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = ""


## 1. ChatOpenAI Agent 생성


gpt-4o-mini 모델을 사용하는 ChatOpenAI Agent를 생성합니다. 
- ChatOpenAI Agent는 사용자의 입력을 Ollama를 통해 로컬에서 구동한 LLM에 전송하고, 그 답변을 반환합니다.
- 본 RAG 과정에서는 LLM으로 ChatOpenAI를 활용할 것입니다.

In [ ]:
# 먼저, gpt-4o-mini 모델을 사용하는 ChatOpenAI 객체를 생성합니다.
llm = ChatOpenAI(model="gpt-4o-mini", api_key="")

Agent를 구성했으니, 이제 Agent를 사용해봅시다.

### 1-1. Runnable interface

LangChain에서 Chain으로 엮을 수 있는 대부분의 구성 요소 (Agent, Tool 등..)는 "Runnable" protocol을 공유합니다.
- 관련 LangChain API 문서: [langchain_core.runnables.base.Runnable — 🦜🔗 LangChain 0.1.16](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable)

Runnable protocol을 공유하는 구성 요소는 모두 아래 세 메서드를 가지고 있습니다:
- stream: 구성 요소의 답변을 순차적으로 반환한다 (stream back)
- invoke: 입력된 값으로 chain을 호출하고, 그 결과를 반환한다.
- batch: 입력값 리스트 (batch)로 chain을 호출하고, 그 결과를 반환한다.

예시로, 저희가 방금 사용한 `ChatOpenAI` Class는 "Runnable" 하기 때문에 `invoke` 메서드를 가지고 있습니다.
- invoke() 메서드를 통해 Agent, Chain 등에 데이터를 입력하고, 그 출력을 받아올 수 있습니다.

`invoke` 메서드를 사용해봅시다. 여기서는 "당신은 누구입니까?" 라는 질문을 입력하면 Agent가 OpenAI API를 통해 Mistral 7B 모델의 답변을 받아 출력할 것입니다.

In [ ]:
llm.invoke("당신은 누구입니까?")

단순 텍스트 뿐만 아니라, 시스템, 사람, AI의 답변을 리스트로 정리하여 입력할 수 있습니다. 

여기서는 LangChain의 `SystemMessage`, `HumanMessage` Class를 활용해봅시다.

In [5]:
messages = [
    SystemMessage("당신은 친절한 AI 어시스턴트 입니다."),
    HumanMessage("당신을 소개해주세요."),
]

response = llm.invoke(messages)

시스템 프롬프트에 '친절한 AI 어시스턴트' 라는 역할을 명시하였습니다.

이제 gpt-4o-mini 모델이 아까와 같은 질문에 어떻게 답했는지 확인해봅시다.

In [ ]:
response

같은 질문을 했음에도 자신을 소개하는 문구가 조금 달라진 것 을 확인할 수 있습니다.

### [TODO] 다양한 역할을 적용해서 어떻게 답변이 달라지는지 자유롭게 실험해보세요.

In [7]:
role = _____________
messages = [
    SystemMessage(_____________),
    HumanMessage(_____________),
]

response = llm.invoke(messages)

In [ ]:
response

## 챗봇 Chain 구성

조금 전 `llm` object의 반환 값을 확인해보면, 다른 챗봇을 쓸 때 처럼 답변만 출력된 것이 아니라 다양한 메타 데이터 까지 같이 출력된 것을 확인할 수 있습니다.

저희가 ChatGPT를 쓸 때를 생각해보면, 챗봇에 이걸 그대로 출력하는건 좀 부자연스럽습니다.

이를 방지하기 위해, 답변을 parsing하는 `StrOutputParser`를 실습 2에서 활용해봅시다.